In [1]:
import re
import numpy as np

from collections import Counter

In [2]:
max_annotators = 5

In [3]:
with open('official-2014.combined-withalt.m2') as fr:
    lines = fr.readlines()

In [20]:
def annotation_stat(correction_type=None):
    
    agreed_annotations = np.full((5, 5), 0.000001) # avoid zero-devision
    total_annotations = np.full((5, 5), 0.000001) # avoid zero-devision
    sentence_annotations = []
    
    def calculate_stat_for_sentence():
        annotators_counter = Counter()
        
        # count all coincidences
        for a in sentence_annotations:
            annotators_counter[a[0]] += 1
            for b in sentence_annotations:
                if a[1] == b[1]:
                    agreed_annotations[a[0]][b[0]] += 1
        
        # count all corrections, devided by because we symetrically add similar value
        for a_id, a_annotations in annotators_counter.items():
            for b_id, b_annotations in annotators_counter.items():
                total_annotations[a_id][b_id] += (a_annotations + b_annotations)/2
                 
    
    for line in lines:
        if line.startswith('S'):
            calculate_stat_for_sentence()
            sentence_annotations.clear()

        elif line.startswith('A'):
            annotator = int(line[-2:-1])
            annotation = line[2:-5]
            corr_type = re.search(r'(?<=\|\|\|).+?(?=\|\|\|)', line).group(0)
            if not correction_type or correction_type == corr_type:
                sentence_annotations.append((annotator, annotation))

    calculate_stat_for_sentence()
    
    ratio = [[agreed_annotations[a][b]/total_annotations[a][b] 
                            for a in range(max_annotators)] 
                            for b in range(max_annotators)]
    
    num = 0
    den = 0
    for a in range(max_annotators - 1):
        for b in range(a + 1, max_annotators):
            num += ratio[a][b] * total_annotations[a][b]
            den += total_annotations[a][b]

    return num / den

In [21]:
%%time 

annotation_stat()

CPU times: user 62.2 ms, sys: 243 µs, total: 62.4 ms
Wall time: 60.2 ms


0.3518977237684415

In [22]:
%%time 

annotation_stat("Vform")

CPU times: user 13.6 ms, sys: 229 µs, total: 13.9 ms
Wall time: 13.5 ms


0.7755102224073288

In [23]:
corrections = {
    "ArtOrDet": "Article or Determiner",
    "Cit": "Citation",
    "Mec": "Punctuation, capitalization, spelling, typos",
    "Nn": "Noun number",
    "Npos": "Noun possesive",
    "Others": "Other errors",
    "Pform": "Pronoun form",
    "Pref": "Pronoun reference",
    "Prep": "Preposition",
    "Rloc-": "Local redundancy",
    "SVA": "Subject-verb-agreement",
    "Sfrag": "Fragment",
    "Smod": "Dangling modifier",
    "Spar": "Parallelism",
    "Srun": "Runons, comma splice",
    "Ssub": "Subordinate clause",
    "Trans": "Link word/phrases",
    "Um": "Unclear meaning (cannot be corrected)",
    "V0": "Missing verb",
    "Vform": "Verb form",
    "Vm": "Verb modal",
    "Vt": "Verb tense",
    "WOadv": "Adverb/adjective position",
    "WOinc": "Incorrect sentence form",
    "Wa": "Acronyms",
    "Wci": "Wrong collocation/idiom",
    "Wform": "Word form",
    "Wtone": "Tone"
}

In [19]:
for (corr_type, desc) in corrections.items():
    print(f'{annotation_stat(corr_type):.2}\t{corrections[corr_type]}')

0.63	Article or Determiner
1.0	Citation
0.68	Punctuation, capitalization, spelling, typos
0.8	Noun number
0.79	Noun possesive
0.68	Other errors
0.88	Pronoun form
0.68	Pronoun reference
0.72	Preposition
0.67	Local redundancy
0.85	Subject-verb-agreement
1.0	Fragment
1.0	Dangling modifier
1.0	Parallelism
0.78	Runons, comma splice
0.77	Subordinate clause
0.6	Link word/phrases
0.63	Unclear meaning (cannot be corrected)
0.86	Missing verb
0.78	Verb form
0.59	Verb modal
0.77	Verb tense
0.89	Adverb/adjective position
0.78	Incorrect sentence form
1.0	Acronyms
0.48	Wrong collocation/idiom
0.73	Word form
0.83	Tone
